In [1]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML
import scipy

In [2]:
!ls ../../ppc/scripts

autotune_12000.sh
autotune_eval_10000.sh
autotune_eval_12000_seed.sh
autotune_eval_12000.sh
autotune_eval_8000.sh
autotune_eval_zifa_A.sh
autotune_eval_zifa_B.sh
autotune_eval_zifa_C_new.sh
autotune_eval_zifa_C.sh
autotune_eval_zifa_D_new.sh
autotune_eval_zifa_D.sh
autotune_eval_zifa_F_new.sh
autotune_eval_zifa_F.sh
autotune_eval_zifa_F_sos10.sh
autotune_eval_zifa_G_new.sh
autotune_eval_zifa_G.sh
autotune_eval_zifa_H.sh
autotune_eval_zifa_I.sh
autotune.sh
autotune_zi_12000.sh
autotune_zifa_new.sh
autotune_zifa.sh
autotune_zi.sh
aws_sos
brain_small
brain_small_nb_1200_50_results.json
brain_small_old
brain_small_zinb_1200_50_results.json
brain_small.zip
catchup_new.txt
catchup.txt
check_hemato.py
cortex
cortex_bis
cortex_nb_1200_50_results.json
cortex_old
cortex_zinb_1200_50_results.json
cortex.zip
data
hemato
hemato_nb_1200_50_results.json
hemato_old
hemato_zinb_1200_50_results.json
hemato_zinb_1200_50_results_new.json
hemato.zip
klein_dataset
klein_dataset_nb_1200_50_results.json
klein

In [35]:


data_path = '../../ppc/scripts'
dataset_names = {'cortex': 'Cortex',
                 'hemato': 'Hemato',
                 'brain_small': 'Brain Small',
                 'pbmc': 'PBMC',
                 'retina': 'Retina'}

dataset_names = {'klein_dataset': 'Klein et al. 2015',
                 'macos_dataset': 'Macosko et al. 2015',
                 'zheng_dataset': 'Zheng et al. 2017',
                 'sven1_dataset': 'Svensson et al. 2017 (1)',
                 'sven2_dataset': 'Svensson et al. 2017 (2)'}
data_files = {}
data_names = {}
for dataset_name in dataset_names:
    data_files[dataset_name] = sorted(glob.glob(os.path.join(data_path, '{}/*.p'.format(dataset_name))))
    data_names[dataset_name] = []
    for data_file in data_files[dataset_name]:
        data_name = data_file.split('/')[-1].split('_')[0]
        data_names[dataset_name].append(data_name)

for dataset_name in list(dataset_names.keys()):
    if len(data_names[dataset_name]) != 2:
        del dataset_names[dataset_name]
        del data_files[dataset_name]
        del data_names[dataset_name]

In [36]:
dataset_names

{'klein_dataset': 'Klein et al. 2015',
 'macos_dataset': 'Macosko et al. 2015',
 'zheng_dataset': 'Zheng et al. 2017',
 'sven1_dataset': 'Svensson et al. 2017 (1)',
 'sven2_dataset': 'Svensson et al. 2017 (2)'}

In [37]:
data_names

{'klein_dataset': ['nb', 'zinb'],
 'macos_dataset': ['nb', 'zinb'],
 'zheng_dataset': ['nb', 'zinb'],
 'sven1_dataset': ['nb', 'zinb'],
 'sven2_dataset': ['nb', 'zinb']}

In [38]:
df = {}
for dataset_name in dataset_names:
    dfs = []
    for data_name, f in zip(data_names[dataset_name], data_files[dataset_name]):
        if data_name != 'zifa_full':

            #my_df = pd.read_csv(f, sep='\t')
            my_df = pd.read_pickle(f)
            my_df.loc[:, 'model'] = data_name
            dfs.append(my_df)
            print(data_name, f)
    #         display(my_df)
    df[dataset_name] = pd.concat(dfs, axis=0)

nb ../../ppc/scripts/klein_dataset/nb_klein_dataset.p
zinb ../../ppc/scripts/klein_dataset/zinb_klein_dataset.p
nb ../../ppc/scripts/macos_dataset/nb_macos_dataset.p
zinb ../../ppc/scripts/macos_dataset/zinb_macos_dataset.p
nb ../../ppc/scripts/zheng_dataset/nb_zheng_dataset.p
zinb ../../ppc/scripts/zheng_dataset/zinb_zheng_dataset.p
nb ../../ppc/scripts/sven1_dataset/nb_sven1_dataset.p
zinb ../../ppc/scripts/sven1_dataset/zinb_sven1_dataset.p
nb ../../ppc/scripts/sven2_dataset/nb_sven2_dataset.p
zinb ../../ppc/scripts/sven2_dataset/zinb_sven2_dataset.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=F

In [80]:
from statsmodels.stats.weightstats import ttest_ind

metrics = ['ll_ll', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']

#h1_hypothesis = ['larger', 'larger', 'larger', 'larger', 'larger']
h1_hypothesis = ['greater', 'greater', 'greater', 'greater']

h1_hypothesis_bis = ['smaller', 'smaller', 'smaller', 'smaller']

df_summary = {}

n_to_keep = 50

for dataset_name in dataset_names:

    df_nb = df[dataset_name].loc[df[dataset_name].model=='nb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
    df_zinb = df[dataset_name].loc[df[dataset_name].model=='zinb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
    
    df_nb_2 = df_nb.copy()
    df_nb_2['model'] = 'nb'
    df_zinb_2 = df_zinb.copy()
    df_zinb_2['model'] = 'zinb'
    df_new = pd.concat([df_nb_2, df_zinb_2])
    
    print("Shapes")
    print(df_nb.shape)
    print(df_zinb.shape)
    print(df_new.shape)
    print(df_new)

    def get_pvals(gby, other_df):
        my_df = gby[metrics]
        print(gby)
        assert my_df.shape[1] == len(metrics)
        assert other_df.shape[1] == len(metrics)
        pvals = []
        for idx, alternative in enumerate(h1_hypothesis):
            assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
            #_, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
            try:
                _, pval = scipy.stats.mannwhitneyu(np.sort(other_df.values[:, idx])[:n_to_keep],
                                       np.sort(my_df.values[:, idx])[:n_to_keep][::-1],
                                       use_continuity=True,
                                       alternative=alternative)
            except ValueError:
                pval = 0.5
            
            pvals.append(pval)
        return np.array(pvals)

    pvals_against_zinb = df_new.groupby('model').apply(get_pvals, other_df=df_zinb)
    pvals_against_zinb = (pvals_against_zinb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    pvals_against_nb = df_new.groupby('model').apply(get_pvals, other_df=df_nb)
    pvals_against_nb = (pvals_against_nb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    def get_summary(gby):
        res = {}
        res['median'] = gby.median()
        res['pvals_against_nb'] = pvals_against_nb[gby.name]
        res['pvals_against_zi'] = pvals_against_zinb[gby.name]
        return pd.DataFrame(res).T

    # df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
    #                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
    df_summary[dataset_name] = df[dataset_name].groupby('model')[metrics].apply(get_summary)


    df_summary[dataset_name] = df_summary[dataset_name].stack().unstack(1).sort_index(level=1).swaplevel()

Shapes
(50, 4)
(50, 4)
(100, 5)
         ll_ll  t_dropout_ks_stat  t_ratio_ks_stat  t_cv_ks_stat model
22  119.620654           0.230769         0.192308      0.365385    nb
2   120.209020           0.250000         0.211538      0.384615    nb
44  120.250767           0.211538         0.211538      0.423077    nb
6   120.273103           0.211538         0.173077      0.403846    nb
49  120.532429           0.211538         0.192308      0.365385    nb
43  120.721825           0.230769         0.173077      0.423077    nb
48  120.821391           0.230769         0.173077      0.442308    nb
31  120.837254           0.269231         0.192308      0.442308    nb
36  120.918196           0.250000         0.192308      0.442308    nb
40  120.936232           0.211538         0.211538      0.461538    nb
9   120.967290           0.211538         0.211538      0.442308    nb
20  121.192730           0.250000         0.192308      0.442308    nb
17  121.277359           0.250000         0.1

         ll_ll  t_dropout_ks_stat  t_ratio_ks_stat  t_cv_ks_stat model
4    93.875860           0.277778         0.194444      0.694444    nb
20   94.893681           0.250000         0.222222      0.694444    nb
47   95.724259           0.333333         0.277778      0.694444    nb
49   95.767921           0.222222         0.166667      0.583333    nb
36   95.849160           0.333333         0.250000      0.611111    nb
7    95.933217           0.305556         0.277778      0.666667    nb
0    96.144711           0.250000         0.250000      0.750000    nb
23   96.179697           0.305556         0.194444      0.750000    nb
10   96.308192           0.333333         0.250000      0.722222    nb
1    96.714042           0.305556         0.277778      0.722222    nb
8    97.032950           0.333333         0.277778      0.722222    nb
18   97.204585           0.333333         0.277778      0.777778    nb
17   97.361230           0.361111         0.250000      1.000000    nb
21   9

# New export Latex

In [81]:
def boldizer(row):
    to_bold = (row.pvals_against_nb <= 5e-2) | (row.pvals_against_zi <= 5e-2)
#     print(to_bold)
    if to_bold:
        row.latex = '\mathbf{' + row.latex + '}'
    return row

In [82]:
df_summary

{'klein_dataset':                              median  pvals_against_nb  pvals_against_zi
                   model                                                
 ll_ll             nb     123.215915          0.501375      1.560759e-02
                   zinb   125.120232          0.984661      5.013753e-01
 t_cv_ks_stat      nb       0.442308          0.501402      8.643182e-04
                   zinb     0.500000          0.999156      5.013826e-01
 t_dropout_ks_stat nb       0.250000          0.501396      1.732552e-07
                   zinb     0.288462          1.000000      5.013975e-01
 t_ratio_ks_stat   nb       0.211538          0.501388      1.176819e-03
                   zinb     0.230769          0.998850      5.013898e-01,
 'macos_dataset':                              median  pvals_against_nb  pvals_against_zi
                   model                                                
 ll_ll             nb     375.753590          0.501375      9.998503e-01
                

In [83]:
n_dec = 3

work_df = {}

for dataset_name in dataset_names:

    work_df[dataset_name] = (df_summary[dataset_name]
               .reset_index(level=-1)
               .reset_index()
               .assign(latex=lambda x: x['median'].round(n_dec).astype(str))
               .apply(boldizer, axis=1)
               .assign(Metric=lambda x: 
                       x.loc[:, 'index'].replace({
                           'll_ll': 'Negative LL',
                           't_cv_ks_stat': 'Coefficient of variation',
                           't_dropout_ks_stat': 'Dropout ratio',
                           't_ratio_ks_stat': 'Zeros-to-expression ratio',
                           'model': 'Model'
               }),
                       latex=lambda x: '$' + x.latex + '$'
                      )
              )

    work_df[dataset_name] = work_df[dataset_name].rename(columns={'model': 'Model', 'index': 'metric',
                                                                 'Metric': 'Metric (medians)'})
    work_df[dataset_name]['Model'] = work_df[dataset_name]['Model'].str.upper()
    #work_df[dataset_name]['Dataset'] = dataset_names[dataset_name]
    

In [84]:
work_df.keys()

dict_keys(['klein_dataset', 'macos_dataset', 'zheng_dataset', 'sven1_dataset', 'sven2_dataset'])

In [85]:
dataset_names

{'klein_dataset': 'Klein et al. 2015',
 'macos_dataset': 'Macosko et al. 2015',
 'zheng_dataset': 'Zheng et al. 2017',
 'sven1_dataset': 'Svensson et al. 2017 (1)',
 'sven2_dataset': 'Svensson et al. 2017 (2)'}

In [86]:
pivot = {}
for dataset_name in dataset_names:
    pivot[dataset_name] = pd.pivot(work_df[dataset_name], index='Metric (medians)', values='latex', columns='Model')
    pivot[dataset_name]['Dataset'] = dataset_names[dataset_name]
    pivot[dataset_name].reset_index(inplace=True)
    pivot[dataset_name].set_index(['Dataset', 'Metric (medians)'], inplace=True)
pivot = pd.concat(list(pivot.values()), axis=0)

In [87]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 20 entries, (Klein et al. 2015, Coefficient of variation) to (Svensson et al. 2017 (2), Zeros-to-expression ratio)
Data columns (total 2 columns):
NB      20 non-null object
ZINB    20 non-null object
dtypes: object(2)
memory usage: 553.0+ bytes


In [88]:
res = pivot.to_latex(escape=False)
res_new = res

In [89]:
print(res_new)

\begin{tabular}{llll}
\toprule
                         & Model &                  NB &                ZINB \\
Dataset & Metric (medians) &                     &                     \\
\midrule
Klein et al. 2015 & Coefficient of variation &    $\mathbf{0.442}$ &               $0.5$ \\
                         & Dropout ratio &     $\mathbf{0.25}$ &             $0.288$ \\
                         & Negative LL &  $\mathbf{123.216}$ &            $125.12$ \\
                         & Zeros-to-expression ratio &    $\mathbf{0.212}$ &             $0.231$ \\
Macosko et al. 2015 & Coefficient of variation &    $\mathbf{0.702}$ &             $0.827$ \\
                         & Dropout ratio &    $\mathbf{0.346}$ &             $0.654$ \\
                         & Negative LL &           $375.754$ &  $\mathbf{343.965}$ \\
                         & Zeros-to-expression ratio &             $0.519$ &              $0.51$ \\
Zheng et al. 2017 & Coefficient of variation &    $\mathbf{0.702}$ &    

In [20]:
# import io #Used as buffer


# def convertToLaTeX(df, alignment="c"):
#     """
#     Convert a pandas dataframe to a LaTeX tabular.
#     Prints labels in bold, does not use math mode
#     """
#     numColumns = df.shape[1]
#     numRows = df.shape[0]
#     output = io.StringIO()
#     colFormat = ("%s|%s" % (alignment, alignment * numColumns))
#     #Write header
#     output.write("\\begin{tabular}{%s}\n" % colFormat)
#     columnLabels = ["\\textbf{%s}" % label for label in df.columns]
#     output.write("& %s\\\\\\hline\n" % " & ".join(columnLabels))
#     #Write data lines
#     for i in range(numRows):
#         output.write("\\textbf{%s} & %s\\\\\n"
#                      % (df.index[i], " & ".join([str(val) for val in df.ix[i]])))
#     #Write footer
#     output.write("\\end{tabular}")
#     return output.getvalue()

# convertToLaTeX(pivot)

In [170]:
path = os.path.join(data_path, '{}.tex'.format(dataset_name))
with open(path, 'w') as tf:
     tf.write(res)

In [58]:
path

'../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_10.0.tex'